In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# Part 1: Outcomes from No-regret Learning in Games

## Set Up and Algorithm Definition

In [143]:
class EWAlg:
    def __init__(self, epsilon, k, h, myBids, myValue):
        self.weights = np.ones(k)
        self.payoffs = np.zeros(k)
        self.h = h
        self.k = k
        self.epsilon = epsilon
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        self.myValue = myValue
        self.myBids = myBids
        self.myTotalValue = 0

    def getBids(self):
        return self.myBids

    def getValue(self):
        return self.myValue

    def getTotalValue(self):
        return self.myTotalValue

    def getAction(self):
        j = np.random.choice(self.k, 1, p = self.probs)
        return self.myBids[j.item()]
    
    def update(self, payoffs):
        for j in range(len(payoffs)):
            curPayoff = payoffs[j]
            self.payoffs[j] = self.payoffs[j] + curPayoff
            newWeight = (1+self.epsilon)**(self.payoffs[j]/self.h)
            self.weights[j] = newWeight
        self.sumWeights = np.sum(self.weights)
        if(self.epsilon > 1):
            self.weights = self.weights/self.sumWeights
            self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        return

    def generatePayoffs(self, winningBid, meWin):
        if meWin:
            self.myTotalValue += self.myValue - winningBid
        payoffs = np.zeros(len(self.myBids))
        for count, bid in enumerate(self.myBids):
            if bid >= winningBid:
                payoffs[count] = bid - self.myValue
            else:
                payoffs[count] = 0
        return payoffs
    
    def reset(self):
        self.weights = np.ones(self.k)
        self.payoffs = np.zeros(self.k)
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        self.myTotalValue = 0


In [140]:
class FirstPriceReserve:
    # bidders have to be ordered the same way every time
    # reserve given as a raw value
    def __init__(self, numBidders, reserve = 0):
        self.numBidders = numBidders
        self.totalPayoffs = np.zeros(self.numBidders)
        self.reserve = reserve
        
    def generate(self, bids):
        winningBid = 0
        winner = -1
        tiedBidders = []
        tied = False
        # check bids of all bidders
        for count, bid in enumerate(bids):
            if bid > winningBid:
                winningBid = bid
                winner = count
            elif (bid == winningBid) and (winner != -1):
                tiedBidders.append(count)
                tied = True
        if tied : winner = random.choice(tiedBidders)
        # see if winning bid is greater than reserve price
        if (winningBid < self.reserve): 
            winningBid = self.reserve
            winner = -1
        return winningBid, winner

In [141]:
numPlayers = 2
epsilons = [2,5]
h = 1
k = 20
stepSize = 0.15

def setUpPlayers() :
    # set up our players
    players = []
    for count in range(numPlayers):
    # pick a distribution and numPlayers values from it
        playerValue = random.uniform(0,h)
        # create possible bids using geometric discretization
        playerBids = []
        for j in range(k):
            playerBids.append(playerValue - ((1 + stepSize)**(-j))*playerValue)
        # create player
        player = EWAlg(epsilons[count], k, h, playerBids, playerValue, )
        players.append(player)
    return players

def conductAuction(players, auction, n = 100):
    # do the action n times
    allBids = []
    allWinners = []
    for i in range(n):
        bids = []
        # generate bids
        for count, player in enumerate(players):
            bids.append(player.getAction())
        # conduct auction
        winningBid, winner = auction.generate(bids)
        # update payoffs
        for count, player in enumerate(players):
            if count == winner:
                payoffs = player.generatePayoffs(winningBid, True)
            else:
                payoffs = player.generatePayoffs(winningBid, False)
            player.update(payoffs)
        allBids.append(bids)
        allWinners.append(winner)
    return allBids, allWinners

In [133]:
k = 20
playerBids = []
playerValue = random.uniform(0,1)
stepSize = 0.15
print(playerValue)
print(stepSize)
for j in range(k):
    playerBids.append(playerValue - ((1 + stepSize)**(-j))*playerValue)
print(playerBids)

0.5038244486512475
0.15
[0.0, 0.06571623243277136, 0.12286078237431175, 0.17255169536695553, 0.21576118492577623, 0.2533346541073594, 0.28600723600438827, 0.31441817678441336, 0.33912334268008737, 0.3606060956328474, 0.3792867503743778, 0.39553079797570867, 0.4096560567594746, 0.4219388904844884, 0.43261961546276134, 0.44190720240039, 0.44998336495484964, 0.4570061150022059, 0.46311285417382003, 0.46842306214913665]


In [144]:
MCBound = 500
n = 100
reserveFrac = 0 #between 0 and 1
reserveValue = reserveFrac * h
# set up players
players = setUpPlayers()
# set up the auction 
auction = FirstPriceReserve(2, reserveValue)

values = []
totalWinnings = [[], []]
possibleBids = []
for player in players:
    values.append(player.getValue())
    possibleBids.append(player.getBids())

MCBids = []
MCWinners = []
for i in range(MCBound):
    # conduct auction
    allBids, allWinners = conductAuction(players, auction, n)
    for count, player in enumerate(players):
        totalWinnings[count].append(player.getTotalValue())
    # reset players -> they keep same learning rate and values
    for player in players:
        player.reset()
    MCBids.append(allBids)
    MCWinners.append(allWinners)
avgBids = np.array(MCBids)
avgBids = np.mean(avgBids, 0)
totalWinnings = np.mean(np.array(totalWinnings), 1)

print(values)
print(totalWinnings)
print(avgBids)

[0.648696632370703, 0.12361666759623624]
[6.63197497 1.7953271 ]
[[0.4197478  0.07915517]
 [0.41334311 0.0785462 ]
 [0.41583266 0.08038906]
 [0.40970105 0.0800967 ]
 [0.40799154 0.07938007]
 [0.39959302 0.08257138]
 [0.39784578 0.0773167 ]
 [0.40575558 0.08129788]
 [0.3967053  0.07910322]
 [0.39993841 0.08004943]
 [0.39230364 0.07903051]
 [0.40015593 0.07939445]
 [0.40579976 0.07872292]
 [0.39087983 0.08159831]
 [0.39763624 0.07696568]
 [0.38603773 0.0790308 ]
 [0.38250983 0.07788786]
 [0.38321423 0.07840694]
 [0.35354533 0.07580554]
 [0.3727526  0.07581354]
 [0.35056761 0.07484582]
 [0.36891068 0.0792439 ]
 [0.35795028 0.077139  ]
 [0.39251107 0.08053423]
 [0.3631634  0.07856163]
 [0.36700199 0.07911875]
 [0.33003148 0.07795509]
 [0.34997296 0.07792581]
 [0.35417069 0.07844693]
 [0.32797321 0.08009842]
 [0.32906845 0.07791665]
 [0.31373644 0.0773498 ]
 [0.34125043 0.08047029]
 [0.33248379 0.07828166]
 [0.31585858 0.07809854]
 [0.31991506 0.07884097]
 [0.30297792 0.07798124]
 [0.299518

In [ ]:
print(MCBids)

In [3]:
def MonteCarlo(numTrials, payoffGenerator, epsilon, k, h, n):
    avgFinalPayoff = 0
    avgRegretPerRound = [[] for i in range(numTrials)]
    for trial in range(numTrials):
        alg = EWAlg(epsilon, k, h)
        finalPayoff = 0
        actionPayoffs = np.zeros(k)
        generator = payoffGenerator(k)
        regretPerRound = np.zeros(n)
        for i in range(n):
            payoffs = generator.generate()
            j = alg.getAction()
            myPayoff = payoffs[j]
            actionPayoffs += payoffs
            alg.update(payoffs)
            finalPayoff += myPayoff
            OPT = max(actionPayoffs)
            regret = (OPT - finalPayoff).item() / (i+1)
            regretPerRound[i] = regret
        avgFinalPayoff += finalPayoff
        avgRegretPerRound[trial] = regretPerRound
    return avgFinalPayoff/numTrials, np.mean(avgRegretPerRound, axis=0)

In [3]:
def MonteCarloTrackActions(numTrials, payoffGenerator, epsilon, k, h, n):
    avgFinalPayoff = 0
    avgRegretPerRound = [[] for i in range(numTrials)]
    actionTrial = []
    for trial in range(numTrials):
        alg = EWAlg(epsilon, k, h)
        finalPayoff = 0
        actionPayoffs = np.zeros(k)
        generator = payoffGenerator(k)
        regretPerRound = np.zeros(n)
        actions = np.zeros(n)
        for i in range(n):
            payoffs = generator.generate()
            j = alg.getAction()
            actions[i] = j
            myPayoff = payoffs[j]
            actionPayoffs += payoffs
            alg.update(payoffs)
            finalPayoff += myPayoff
            OPT = max(actionPayoffs)
            regret = (OPT - finalPayoff).item() / (i+1)
            regretPerRound[i] = regret
        actionTrial.append(actions)
        avgFinalPayoff += finalPayoff
        avgRegretPerRound[trial] = regretPerRound
    return avgFinalPayoff/numTrials, np.mean(avgRegretPerRound, axis=0), np.array(actionTrial)

## Adversarial Fair Payoffs

In each round i:

Draw a payoff x ~ U[0,1] (i.e., from the uniform distribution on interval [0,1])

Assign this payoff to the action j* that has the smallest total payoff so far, i.e., j* = argminj Vji-1 where Vji = Σir=1 vji. 
(All other actions get 0 payoff in round i.)

In [202]:
h = 1 # fixed
# hyperparameters
k = 10
n = 100
epsilons = [-0.99, -0.5, 0, 0.1, 0.1517, 0.2, 0.4, 0.7, 1, 2, 10, 1000] # to be studied
monteCarloBound = 1000

AFEpsilonPayoffs = []
AFEpsilonRegretPerRound = []
for epsilon in epsilons:
    finalPayoff, regretPerRound = MonteCarlo(monteCarloBound, AdversarialFair, epsilon, k, h, n)
    AFEpsilonPayoffs.append(finalPayoff)
    AFEpsilonRegretPerRound.append(regretPerRound)

AFEpsilonAvgRegrets = [i[99] for i in AFEpsilonRegretPerRound]
print(epsilons)
print(AFEpsilonPayoffs)
print(AFEpsilonAvgRegrets)

[-0.99, -0.5, 0, 0.1, 0.1517, 0.2, 0.4, 0.7, 1, 2, 10, 1000]
[12.572066468624602, 6.089115420080595, 4.952127590364313, 4.888985005026019, 4.662385648248368, 4.658095671678926, 4.491083295527148, 4.237502645046786, 3.988782430758382, 3.551178754492665, 2.1475778776821812, 0.3206374990307352]
[-0.07164880805723829, -0.0066178097291542375, 0.0044772387027638095, 0.005445673155832275, 0.007507744684371933, 0.007499525986132154, 0.0091217183309684, 0.011695349045173858, 0.014306945820161048, 0.01850769358056152, 0.03260957846953832, 0.05091754046141372]
